### Importing libraries

In [ ]:
import cv2
import time
import logging
import nest_asyncio
import math
import sys
from arduino_iot_cloud import ArduinoCloudClient
import glob
from pathlib import Path

### Web cam setup and basic image filtering (droid cam)

In [ ]:
cam = cv2.VideoCapture('Webcam IP goes here')

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k % 256 == 27:
        
        print("Escape hit, closing...")
        break
    elif k % 256 == 32:
        img_name = 'Images/opencv_frame_{}.png'.format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()
cv2.destroyAllWindows()

folder_dir = 'Images'  # creating a directory to store the images
images = Path(folder_dir).glob('*.png')

def blur_detect(variance):
    if variance <120:   #arbritrary threshold, kept low because camera quality was an issue
        return 0
    else:
        return 1

for img in images:
    image_temp = cv2.imread(str(img))
    x = int(image_temp.shape[0])
    y = int(image_temp.shape[1])
    image_c = image_temp[50:x, 50:y]
    grey = cv2.cvtColor(image_c, cv2.COLOR_BGR2GRAY)
    var = cv2.Laplacian(grey, cv2.CV_64F).var()
    val = blur_detect(var)
    if val == 1:
        image = image_c
        break
    else:
        image = None

### Image processing and getting the cordinates of the required contours

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #good image goes here
blur = cv2.bilateralFilter(image, 9, 75, 75)
ret, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
morph = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
contours, hierarchies = cv2.findContours(
    morph, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

x_cords = []
y_cords = []
area = []

# for detecting larger outer white circlular contour
for i in contours:
    M = cv2.moments(i)
    if M['m00'] > 100:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        ar = int(M['m00'])
        cv2.drawContours(image, [i], -1
                         , (0, 255, 0), 2)
        cv2.circle(image, (cx, cy), 1, (0, 0, 255), -1)

        x_cords.append(cx)
        y_cords.append(cy)
        area.append(ar)

# for detecting smaller dark circles on the object
rt, th = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
mor = cv2.morphologyEx(th, cv2.MORPH_CLOSE, ker)

conts, her = cv2.findContours(
    mor, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

for i in conts:
    M = cv2.moments(i)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00']) # inbuilt functions, taken directly from the documentation of opencv
        cy = int(M['m01'] / M['m00'])
        ar = int(M['m00'])
        if ar > 20 and ar <= 50:
            cv2.drawContours(image, [i], -1
                             , (0, 255, 0), 2)
            cv2.circle(image, (cx, cy), 1, (0, 0, 255), -1)

            x_cords.append(cx)
            y_cords.append(cy)
        area.append(ar)

cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Quality Check 

In [ ]:
zip_list = zip(area, x_cords, y_cords)
sorted_list = sorted(zip_list, reverse=True)
final_list = [k for k in sorted_list[1:]]

x_cords_final = []
y_cords_final = []
for i in final_list:
    x_cords_final.append(i[1])
    y_cords_final.append(i[2])


def f(lx, ly):
    l = []
    if len(lx) == 1:
        l.append(lx[0]) #if one dark circle, directly append its cords to the list
        l.append(ly[0])
    elif len(lx) == 2:
        l.append((lx[0] + lx[1])/2) #if 2 dark circles, appending the mid point of those two
        l.append((ly[0] + ly[1])/2)
    else:
        return
    return l

center_cords = f(x_cords_final, y_cords_final)
big_circle_cords = sorted_list[0][1:]
big_area = sorted_list[0][0]

radius = math.sqrt(big_area / math.pi)
rad_per = radius * 0.1
    
distance = math.dist(center_cords, big_circle_cords)
def reject_status(dist):
    if dist < rad_per:
        return 0
    else:
        return 1

### Python-Arduino Cloud communication

In [ ]:
# the following code is taken from arduino cloud documentation
nest_asyncio.apply()
sys.path.append("lib")

DEVICE_ID = "secret device id for microcontroller"
SECRET_KEY = "secret key" # we used ESP8266


def logging_func():
    logging.basicConfig(
        datefmt="%H:%M:%S",
        format="%(asctime)s.%(msecs)03d %(message)s",
        level=logging.INFO)


if __name__ == "__main__":
    logging_func()
    client = ArduinoCloudClient(device_id=DEVICE_ID, username=DEVICE_ID, password=SECRET_KEY)

    # reject variable is from the quality check output
    client.register("on_stations")
    client.register("red")
    client.register("green")
    client.register("rejected")
    client.register("reject")
    client["reject"] = 1

    client.start()